<a href="https://colab.research.google.com/github/HoYoungChun/financial_data_analysis/blob/main/NCAV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NCAV(Net Current Asset Value)

### Basic Setting

In [1]:
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
import urllib.request
# !pip install bs4
# !pip install urllib


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import pandas as pd

csv_url = '/content/drive/MyDrive/Colab/financial_data_analysis/company.csv'

company = pd.read_csv(csv_url, encoding='utf-8')

In [4]:
SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
print(SNAP_URL)

https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701


In [5]:
FS_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701'
print(FS_URL)

https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701


In [6]:
sample_code = '035720'

In [11]:
fs = urllib.request.urlopen(FS_URL.format(sample_code)).read()
fs_soup = BeautifulSoup(fs,'html.parser')

In [13]:
snap = urllib.request.urlopen(SNAP_URL.format(sample_code)).read()
snap_soup = BeautifulSoup(snap,'html.parser')

### Market Value(시가총액)

In [29]:
mv_sells = snap_soup.find('div',{'id':"svdMainGrid1"}).find_all('td',{'class':'r'})
market_value = float(mv_sells[8].string.replace(',',''))
market_value

699189.0

### Current Asset(유동자산)

In [33]:
ca_cells = fs_soup.find('tr',{'id':"p_grid2_2"}).find_all('td',{'class':'r cle'})
current_asset = float(ca_cells[0].string.replace(',',''))
current_asset

52079.0

### Total Debt(총부채)

In [40]:
debt_cells = fs_soup.find('div',{'id':"divDaechaY"}).find_all('tr',{'class':'rwf rowBold'})
total_debt = float(debt_cells[1].find('td',{'class':'r cle'}).string.replace(',',''))
total_debt

56369.0

In [46]:
debt_list=[]
for debt in debt_cells[1]:
  if debt != '\n':
    debt_list.append(debt.string)
total_debt = float(debt_list[-1].replace(',',''))
total_debt

56369.0

### Net Income(당기순이익)

In [52]:
income_cells = fs_soup.find('div',{'id':'divSonikY'}).find_all('tr',{'class':'rwf rowBold'})
net_income = float(income_cells[-1].find_all('td',{'class':'r'})[-3].string.replace(',',''))
net_income

2399.0

### Get All Stock code

In [ ]:
code_list = company['종목코드']
sample_code_list = code_list[1000:1050]
sample_code_list

In [59]:
def crawler(li):
    # url을 세팅한다.
    SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
    FS_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701'

    result={}

    for code in tqdm(li):
        try:
            fs = urllib.request.urlopen(FS_URL.format(code)).read()
            fs_soup = BeautifulSoup(fs,'html.parser')

            snap = urllib.request.urlopen(SNAP_URL.format(code)).read()
            snap_soup = BeautifulSoup(snap,'html.parser')

            #get company name
            name = company[company['종목코드'] == code]['회사명'].values[0]

            #get market value
            mv_sells = snap_soup.find('div',{'id':"svdMainGrid1"}).find_all('td',{'class':'r'})
            market_value = float(mv_sells[8].string.replace(',',''))

            #get current asset
            ca_cells = fs_soup.find('tr',{'id':"p_grid2_2"}).find_all('td',{'class':'r cle'})
            current_asset = float(ca_cells[0].string.replace(',',''))

            #get total debt
            debt_cells = fs_soup.find('div',{'id':"divDaechaY"}).find_all('tr',{'class':'rwf rowBold'})
            total_debt = float(debt_cells[1].find('td',{'class':'r cle'}).string.replace(',',''))

            #get net income
            income_cells = fs_soup.find('div',{'id':'divSonikY'}).find_all('tr',{'class':'rwf rowBold'})
            net_income = float(income_cells[-1].find_all('td',{'class':'r'})[-3].string.replace(',',''))

            #insert into result dict
            result[name]=[code, market_value, current_asset, total_debt,net_income]

        except(ValueError, AttributeError, IndexError, TypeError):
            pass

    # convert dict into DataFrame
    result_df = pd.DataFrame(result)

    # transposing DataFrame
    result_df = result_df.transpose()

    # set column names
    result_df.columns=['Code','market_value','current_asset', 'total_debt','net_income']
    
    return result_df
        



In [58]:
result_df = crawler(sample_code_list)
result_df

100%|██████████| 50/50 [01:31<00:00,  1.83s/it]


,Code,market_value,current_asset,total_debt,net_income
퓨쳐스트림네트웍스,214270,1619,1144,1234,-4
포시에스,189690,761,194,18,25
덕산네오룩스,213420,13518,1017,283,123
국일신동,060480,599,192,52,4
바디텍메드,206640,5273,1250,355,123
썸에이지,208640,4391,347,31,140
와이제이엠게임즈,193250,1427,285,174,-13
화이브라더스코리아,204630,997,628,1040,-92
하이셈,200470,1260,198,592,-2
휴메딕스,200670,3595,894,452,40


In [60]:
tmp_df = result_df.copy()

### Searching stocks

In [65]:
tmp_df['NCAV'] = tmp_df['current_asset'] - tmp_df['total_debt']
tmp_df

,Code,market_value,current_asset,total_debt,net_income,Current_asset,NCAV
퓨쳐스트림네트웍스,214270,1619,1144,1234,-4,1234,-90
포시에스,189690,761,194,18,25,18,176
덕산네오룩스,213420,13518,1017,283,123,283,734
국일신동,060480,599,192,52,4,52,140
바디텍메드,206640,5273,1250,355,123,355,895
썸에이지,208640,4391,347,31,140,31,316
와이제이엠게임즈,193250,1427,285,174,-13,174,111
화이브라더스코리아,204630,997,628,1040,-92,1040,-412
하이셈,200470,1260,198,592,-2,592,-394
휴메딕스,200670,3595,894,452,40,452,442


In [68]:
tmp_df[(tmp_df['NCAV'] > tmp_df['market_value']*0.3) & (tmp_df['net_income'] > 0)]

,Code,market_value,current_asset,total_debt,net_income,Current_asset,NCAV
슈피겐코리아,192440,3506,2244,630,168,630,1614
메디아나,041920,1211,558,110,25,110,448
